In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import lower, lit, regexp_replace, col, contains
import os
from custom_utils import *

In [ ]:
spark = SparkSession.\
    builder.\
    appName("preprocessing-notebook").\
    getOrCreate()

In [ ]:
station_files = recursive_file_retrieval(os.path.join(project_base_dir, f"tankerkoenig-data/stations/{year}/"), month_strings)

In [ ]:
schema = StructType() \
      .add("uuid",StringType(),True) \
      .add("name",StringType(),True) \
      .add("brand",StringType(),True) \
      .add("street",StringType(),True) \
      .add("house_number",IntegerType(),True) \
      .add("post_code",IntegerType(),True) \
      .add("city",StringType(),True) \
      .add("latitude",DoubleType(),True) \
      .add("longitude",DoubleType(),True) \
      .add("first_active",StringType(),True) \
      .add("openingtimes_json",StringType(),True)

In [ ]:
all_stations = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load(station_files)

In [ ]:
all_stations.show(10)
print(all_stations.dtypes)
print(all_stations.count())

In [ ]:
lower_brand = "lower_brand"
brand_stations = all_stations \
    .withColumn(lower_brand, lower(all_stations.brand)) \
    .filter(col(lower_brand) == brand) \
    .drop(lower_brand)

In [ ]:
brand_stations.show(10)

In [ ]:
standardised_city = "standardised_city"
standardised_stations = brand_stations \
    .withColumn('lower_city', lower(brand_stations.city)) \
    .withColumn(standardised_city, regexp_replace(col("lower_city"), lit("ü"), lit("ue"))) \
    .drop("lower_city")

In [ ]:
standardised_stations.show(10)

In [ ]:
city_rlike = f"({'|'.join(cities)})"
selected_city_stations = standardised_stations \
    .filter(col(standardised_city).rlike(city_rlike))

In [ ]:
selected_city_stations.show(10)

In [ ]:
# need to remove Frankfurt an der Oder
selected_city_stations_clean = selected_city_stations \
    .where(~(col(standardised_city).contains("frankfurt") & (col("latitude") > 51.5)))

In [ ]:
selected_city_stations_clean.show(10),
print(selected_city_stations_clean.count())

In [ ]:
selected_city_stations_unique = selected_city_stations_clean \
    .dropDuplicates(["uuid"]) \
    .drop("name", "brand", "street", "house_number", "post_code", "city", "first_active", "openingtimes_json")

In [ ]:
n_stations = selected_city_stations_unique.count()
selected_city_stations_unique.show(n_stations)
print(n_stations)

In [ ]:
selected_city_stations_unique.write.csv(os.path.join(project_base_dir, "outputs/selected_stations_unique.csv"))